<a href="https://colab.research.google.com/github/PomesHouse/SkillTreePython-DeepLearning/blob/main/04.Paper-with-code/U_Net_Convolutional_Networks_for_Biomedical_Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-Net: Convolutional Networks for Biomedical Image Segmentation
[원본](https://arxiv.org/pdf/1505.04597.pdf)


---
* 이름: 박수영

참고 

[Medical Image Segmentation SOA](https://paperswithcode.com/task/medical-image-segmentation)

[U-Net 리뷰](https://velog.io/@minkyu4506/%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-U-Net-%EB%A6%AC%EB%B7%B0)

[Review: U-Net (Biomedical Image Segmentation)](https://towardsdatascience.com/review-u-net-biomedical-image-segmentation-d02bf06ca760)

[U-NET Implementation from Scratch using TensorFlow](https://medium.com/geekculture/u-net-implementation-from-scratch-using-tensorflow-b4342266e406)




## 1. Introduction
biomedical image processing tasks에서는 localization이 불가피하고 대량의 training images를 구하기 어렵다. 

Ciresan et al.은 각 픽셀 주위의 local region (즉, patch)를 인풋으로 제공함으로써 그 픽셀의 클래스 라벨을 예측하기 위해 sliding-window setup에서 네트워크를 훈련시켰다. 이 네트워크는 localize 할 수 있고, patches 로서의 training data는 훈련 이미지의 숫보다 훨씬 크다는 장점이 있다. 단점이라 할 것 같으면 느리다는 점과 중복이 너무 많다는 점, 또한 localization 정확도와 context의 사용 사이의 trade-off가 있다는 점이 있다. 패치카 커지면 더 많은 max-pooling layers가 요구되고 이는 localization 정확도를 감소시키는 반면, 패치가 작아진다면 context 정보량이 줄어든다. 
가장 최근의 접근법은 multiple layers 의 features를  고려하는 classifier ouput을 제안했고 좋은 localization과 context의 사용이 동시에 가능하다.

이 논문에서는 아주 적은 수의 training images를 가지고 더 정확한 segmentation을 yield 하기 위해 기존 architecture를 수정하고 연장하였다. 메인 아이디어는 일반적인 contracting network를, pooling 연산자를 upsampling 연산자로 대체한 연속적인 레이어로 보충하는 것이다. 이 레이어들은 output의 해상도를 증가시킨다. localize 하기 위해서 contracting path의 고해상도 features는 upsampled된 ouput과 합쳐진다. 뒤이은 convolution layer는 이 정보를 바탕으로 더 정확한 output을 조합하도록 배울 수 있다. 

이 논문의 구조에서 한가지 중요한 modification은 네트워크가 더 높은 해상도 레이어로 context information 전파를할 수 있게 하기위해 upsampling 부분에서 많은 수의 feature channel을 사용한다는 점이다. 결과적으로 expansive path는 contracting path와 대칭을 이루게되고 u 모양 구조를 가지게 되는 것이다. 이 네트워크는 FC layers를 전혀 사용하지 않고 오로지 각 convolution의 유효한 부분만을 사용한다. 즉, segmentation map은 input image에서 전체 컨텍스트가 제공되는 pixels 만을 포함한다. 이 전략은 `overlap-tile strategy`로 임의적으로 큰 이미지의 seamless segmentation 을 가능하게 한다. 이미지 경계지역의 pixels을 예측하기 위해서, 빠진 context는 input image를 mirroring함을 통해서 추론된다. 이 tiling strategy는 해상도가 GPU 메모리에 의해 제한될 수 있는 큰 images에 네트워크를 적용하는데 중요하다. 
[참고 QnA](https://stats.stackexchange.com/questions/474904/overlap-tile-strategy-in-u-nets)

작업하는데 아주 적은 training data가 제공되기 때문에 우리는 제공되는 training images에 elastic deformations을 적용하는 방식으로 과도한(excessive) data augmentation을 사용했다. 이것은 네트워크가  그런 변형(deformation)에 대해 불변성(invariance)을 배울 수 있게 한다, without
the need to see these transformations in the annotated image corpus. 이것은 biomedical segmentation 에서 특히 중요하다. 왜냐하면, deformation 은 조직(tissue)의 가장 흔한 variation 이고 현실적인 deformation 은 효휼적으로 시뮬레이션 될 수 있기 때문이다. 불변성 학습을 위한 data augmentation 의 이러한 가치는 unsupervised feature learning에서  Dosovitskiy et al 에 의해 보여진바 있다.

* `Elastic deformation` refers to a temporary deformation of a material's shape that is self-reversing after removing the force or load. It alters the shape of a material upon the application of a force within its elastic limit.


많은 cell segmentation 작업의 또 다른 과제는 동일한 클래스의 서로 접촉하는 객체의 분리 이다. 
이 것 때문에, 우리는 **the separating background labels between touching cells 이 loss function에서 큰 weight을 가지는 가중 손실(a weighted loss)의 사용을 제안**한다. The resulting network 는 다양한 biomedical segmentation 문제에 적용될 수 있다. 



## 2. Network Architecture

![](https://lmb.informatik.uni-freiburg.de/Publications/2015/RFB15a/u-net-architecture.png)

네트워크 구조는 크게 두 가지 경로로 구분이 된다. 
1. a contracting path:
        전형적인 conv net의 구조를 따른다. 
        * two 3x3 convlutions (unpadded), each followed by ReLU 
        * one 2x2 max pooling operation w/ stride 2
            * 각각의 downsampling step에서 feature channel의 수를 두배로 증가시킨다. 
2. an expansive path
        feature map의 upsampling
        * one 2x2 convolution ("up-convolution")으로 feature channels의 수를 반으로
        * a concatenation with the correspondingly cropped feature map from the contracting path 
            * 각각의 convolution layer를 거칠때 마다 일어나는 가장자리 pixels의 손실 때문에 contracting path의 feature map을 cropping하는 것은 필수 
        * two 3x3 convolutions (unpadded), each followed by a ReLU 

최종 레이어에서 one 1x1 convolution 이 각각의 64-component feature vector와 desired number of classes를 맵핑하는데 사용되었다. 

* 최종적으로 네트워크는 23개의 convolutional layers를 가지게 된다. 
* A seamless tiling of the oupput segmentation map을 얻기 위해서 모든 2x2 max-pooling operations이 동일한(even) x, y 사이즈로 레이어에 적용될 수 있도록 input tile size를 선택하는 것은 중요하다. 




## 3. Training

* sgd implementatoin of [Caffe](https://arxiv.org/abs/1408.5093?context=cs.CV)
* 큰 batch size 보다는 큰 input tiles을 선택 -> the batch was reduced to a single image
* high momentum (0.99) 사용 -> a large number of the previously seen training samples determine the update in the current optimization step.


* Energy function:
    * computed by a pixel-wise soft-max over the final feature map + the cross entropy loss function

* 훈련 데이터셋에서 특정 클래스로부터의 픽셀의 다른 빈도를 보상하기 위하여, 그리고 네트워크가 인접하는 세포사이에 있는 작은 경계선을 배울 수 있도록 하기 위하여 각각의 ground truth segmentation에 대한 weight map은 사전에 계산이 되었다.  
* 이상적으로 초기 가중치는 네트워크의 feature  map 각각이 대락적인 unit variance를 가질 수 있도록 맞춰져 있어야 한다. 이 논문에 나오는 (convolution 과 ReLU 레이어가 교대로 나오는)네트워크 구조 같은 경우 이것은 표준편차가 sqrt(2/N)인 가우시안 분포에서 초기 가중치를 뽑음으로써  달성될 수 있다.(N: 한 뉴런에 들어오는 노드의 수, 예를 들어 3x3 convolution and 64 feature channels이 이전 레이어라면 N 값은 9·64 = 576)

### 3.1 Data Augmentation

Introduction에서 언급했다시피 

"Data augmentation is essential to teach the network the desired invariance and
robustness properties, when only few training samples are available."
* `머신러닝에서 Robust란?`
머신러닝에서 일반화(generalization)는 일부 특정 데이터만 잘 설명하는(=overfitting) 것이 아니라 범용적인 데이터도 적합한 모델을 의미한다. 즉, 잘 일반화하기 위해서는 이상치나 노이즈가 들어와도 크게 흔들리지 않아야(=robust) 한다. [출처](https://wegonnamakeit.tistory.com/9)


현미경 이미지의 경우 주로 변형 및 회색 값 변동(deformations and gray value variations) 에 대한 robustness 뿐만 아니라 shift and rotation 불변성(invariance) 또한 필요하다. 특히 훈련 샘플의 random elastic deformations는 아주 적은 수의 주석 달린 이미지로  segmentation network를 훈련시키는데 핵심 개념인 것 처럼 보인다. 
Coarse 3x3 그리드에 random displacement vectors를 사용해 smooth deformations을 생성한다. 변위(the displacements)는 10픽셀 표준편차의 가우시안 분포로 부터 샘플링된다. 픽셀당 변위(per-pixel displacements)는 bicubic interpolation을 사용하여 계산된다. Contracting path 끝의 드롭아웃 레이어는 implicit data augmentation을 추가로 수행한다.

## 4. Experiments

1.  `the segmentation of neuronal structures in electron microscopic recordings`: Ciresan et al. 의 sliding-window convolutional network 의 결과보다 상당히 좋았다. 

2. `a cell segmentation task in light microscopic images`: 92%의 평균 IOU("intersection over union")를 달성했고 이 결과는 2등의 83%보다 유의미하게 큰 수치이다.

3.  `“DIC-HeLa”데이터셋`: 77.5%의 평균 IOU 달성했고 이 결과는 2등의 46%보다 유의미하게 큰 수치이다. 


## 5. Conclusion 

u-net 구조는 다양한 biomedical segmentation applications에서 아주 좋은 성능을 낸다. Elastic deformations를 이용한 data augmentation 덕에, 주석이 달린 적은 수의 이미지만을 필요로 하며 아주 적당한 학습시간을 필요로 한다. (본 논문에서는 NVidia Titan GPU (6 GB)로 10시간).

In [ ]:
# "We provide the full Caffe-based implementation and the trained networks."

---
## Implementation

[data](https://www.kaggle.com/datasets/tanlikesmath/the-oxfordiiit-pet-dataset)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Conv2DTranspose, concatenate

In [ ]:
for file in img:
    index = img.index(file)
    path = os.path.join(path1, file)
    single_img = Image.open(path).convert('RGB')
    single_img = single_img.resize((i_h,i_w))
    single_img = np.reshape(single_img,(i_h,i_w,i_c)) 
    single_img = single_img/256.
    X[index] = single_img
        
    single_mask_ind = mask[index]
    path = os.path.join(path2, single_mask_ind)
    single_mask = Image.open(path)
    single_mask = single_mask.resize((m_h, m_w))
    single_mask = np.reshape(single_mask,(m_h,m_w,m_c)) 
    single_mask = single_mask - 1 
    y[index] = single_mask

1. Encoder

In [ ]:
def EncoderMiniBlock(inputs, n_filters=32, dropout_prob=0.3, max_pooling=True):
    conv = Conv2D(n_filters, 
                  3,  # filter size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(inputs)
    conv = Conv2D(n_filters, 
                  3,  # filter size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(conv)
  
    conv = BatchNormalization()(conv, training=False)
    if dropout_prob > 0:     
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)
    if max_pooling:
        next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv)    
    else:
        next_layer = conv
    skip_connection = conv    
    return next_layer, skip_connection

2. Decoder

In [ ]:
def DecoderMiniBlock(prev_layer_input, skip_layer_input, n_filters=32):
    up = Conv2DTranspose(
                 n_filters,
                 (3,3),
                 strides=(2,2),
                 padding='same')(prev_layer_input)
    merge = concatenate([up, skip_layer_input], axis=3)
    conv = Conv2D(n_filters, 
                 3,  
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(merge)
    conv = Conv2D(n_filters,
                 3, 
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(conv)
    return conv